# Население, пользующееся как минимум базовыми услугами санитарии (%)

In [1]:
import pandas as pd

from helpers import (
    get_data_with_full_locations_and_years, 
    get_omissions,
)

from warnings import simplefilter
simplefilter('ignore')

Эти данные можно найти на сайте ВОЗ по ссылке

https://www.who.int/data/gho/data/indicators/indicator-details/GHO/population-using-at-least-basic-sanitation-services-(-)

In [2]:
# Загрузим таблицу
data = pd.read_csv('../data/who_sanitation.csv')

data.head(2)

,IndicatorCode,Indicator,ValueType,ParentLocationCode,ParentLocation,Location type,SpatialDimValueCode,Location,Period type,Period,...,FactValueUoM,FactValueNumericLowPrefix,FactValueNumericLow,FactValueNumericHighPrefix,FactValueNumericHigh,Value,FactValueTranslationID,FactComments,Language,DateModified
0,WSH_SANITATION_BASIC,Population using at least basic sanitation ser...,text,EUR,Europe,Country,AND,Andorra,Year,2020,...,NaN,NaN,NaN,NaN,NaN,100,NaN,NaN,EN,2023-07-09T21:00:00.000Z
1,WSH_SANITATION_BASIC,Population using at least basic sanitation ser...,text,WPR,Western Pacific,Country,AUS,Australia,Year,2020,...,NaN,NaN,NaN,NaN,NaN,100,NaN,NaN,EN,2023-07-09T21:00:00.000Z


Описание нужных нам столбцов.

- `ParentLocationCode` - код региона
- `SpatialDimValueCode` - код страны
- `Period` - год
- `FactValueNumeric` - население, пользующееся как минимум базовыми услугами санитарии (%)

Оставим только их.

In [3]:
data = data[[
    'ParentLocationCode', 'SpatialDimValueCode', 'Period', 'FactValueNumeric',
]]

data.rename(columns={'FactValueNumeric': 'Sanitation'}, inplace=True)

data.head()

,ParentLocationCode,SpatialDimValueCode,Period,Sanitation
0,EUR,AND,2020,100.00
1,WPR,AUS,2020,100.00
2,EUR,AUT,2020,99.97
3,EMR,BHR,2020,100.00
4,AMR,CHL,2020,100.00


В данной таблице для некоторых стран данные заполнены не за все года.\
Добавим строки с отсутствующими годами, чтобы явно увидеть пропуски.

In [4]:
data = get_data_with_full_locations_and_years(
    data,
    {
        'region_code': 'ParentLocationCode',
        'location_code': 'SpatialDimValueCode',
        'year': 'Period',
    }
)

# Посмотрим на количество пропусков
data.isna().sum()

ParentLocationCode      0
SpatialDimValueCode     0
Period                  0
Sanitation             77
dtype: int64

In [5]:
# Посмотрим на пропущенные данные
omissions_data = get_omissions(data)

omissions_data.sort_values(by=['SpatialDimValueCode', 'Period'])

,ParentLocationCode,SpatialDimValueCode,Period,Sanitation
974,AMR,ARG,2016,NaN
779,AMR,ARG,2017,NaN
584,AMR,ARG,2018,NaN
389,AMR,ARG,2019,NaN
194,AMR,ARG,2020,NaN
...,...,...,...,...
3965,SEAR,TLS,2000,NaN
3770,SEAR,TLS,2001,NaN
576,AMR,VCT,2018,NaN
381,AMR,VCT,2019,NaN


Заполним их средним по региону.

In [6]:
data['Sanitation'] = \
    data.groupby(['ParentLocationCode', 'Period'])['Sanitation'] \
    .transform(lambda x: x.fillna(x.mean()));

# Убедимся, что пропусков нет
data.isna().sum().sum()

0

In [7]:
# Удалим столбцы, которые больше не нужны
data.drop(columns=['ParentLocationCode'], inplace=True)

In [8]:
# Сохраним полученную таблицу для дальнейшего использования
data.to_csv('../data/who_sanitation_prepared.csv', index=False)